# Image Based Visual Servoing



### Import camera calibration data

In [18]:
import pickle
import cv2
import numpy as np
import math
import time
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg

# Load COCO labels
filename = "coco_labels.dat"
filehandler = open(filename, 'rb')
COCO_labels = pickle.load(filehandler)

# Load camera calibration data for undistort
filename = "calibration.dat"
filehandler = open(filename, 'rb')
camera_cal = pickle.load(filehandler)

mtx = camera_cal['mtx']
dist = camera_cal['dist']

### Load pre-trained SSD model

In [3]:
from jetbot import ObjectDetector

model = ObjectDetector('../Notebooks/object_following/ssd_mobilenet_v2_coco.engine')

### Set up camera for object detection
* Camera is set up for 300x300 pixel video
* Use undistort() to undistort camera image prior to object detection

In [2]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

In [19]:
from jetbot import Robot

robot = Robot()

In [358]:
def undistort(img, mtx, dist, crop=False):
    """Undistort camera image based on calibration data"""
    h,  w = img.shape[:2]
    # print (h,w)
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image if asked
    if crop:
        x,y,w,h = roi
        dst = dst[y:y+h, x:x+w]
    return dst

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def create_feature_pts(bbox):
    """ Return 4 feature points of bounding box """
    # extract bounding x and y coordinates
    UL_fp, LR_fp = bbox
    UL_fp_x, UL_fp_y = UL_fp
    LR_fp_x, LR_fp_y = LR_fp
   
    # return UL, UR, LR, LL feature points
    return [(UL_fp_x, UL_fp_y), (LR_fp_x, UL_fp_y), (LR_fp_x, LR_fp_y), (UL_fp_x, LR_fp_y)]
    
def features_errors(currentFeatures, desiredFeatures):
    """ Return feature points error vector """
    UL_current, UR_current, LR_current, LL_current = currentFeatures 
    UL_desired, UR_desired, LR_desired, LL_desired = desiredFeatures 
    
    # calculate error for each feature point
    UL_error = fp_error(UL_current, UL_desired)
    error_vector = np.asarray(UL_error)
    UR_error = fp_error(UR_current, UR_desired)
    error_vector = np.concatenate((error_vector, UR_error), axis=None)
    LR_error = fp_error(LR_current, LR_desired)
    error_vector = np.concatenate((error_vector, LR_error), axis=None)
    LL_error = fp_error(LL_current, LL_desired)
    error_vector = np.concatenate((error_vector, LL_error), axis=None)

    # return 1x8 error vector of UL, UR, LR, LL feature points
    return np.reshape(error_vector, (1,-1))
    
    
def fp_error(current, desired):
    """ Return error of a feature point """
    current_x, current_y = current
    desired_x, desired_y = desired
    return (desired_x-current_x, desired_y-current_y)


def image_Jacobian(fp, mtx, depth):
    """ Generate image jacobiab L for a feature point """
    # focal lengths in pixel unit
    f_u = mtx[0,0]
    f_v = mtx[1,1]
    c_u = mtx[0,2]
    c_v = mtx[1,2]

    # Estimated distance of reference object (m)
    Z = depth

    # Calculate J of feature point
    u_raw, v_raw = fp
    u = u_raw - c_u
    v = v_raw - c_v    
    
    L = np.array([[-f_u/Z, 0, u/Z, u*v/f_u, -(f_u+u*u/f_u), v],
                  [0, -f_v/Z, v/Z, f_v+v*v/f_v, -u*v/f_v, -u]])
    return L

"""
def robot_jacobian(theta):
    
    return np.array([[math.cos(theta),0],
                  [math.sin(theta),0],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,1]])
"""

def robot_jacobian(robot_angle, wheel_radius, axle_length):
    """ calculate the robot jacobian """
    theta = robot_angle
    l = axle_length
    r = wheel_radius
    H = np.array([[math.cos(theta),0],
                  [math.sin(theta),0],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,1]])
    T = np.array([[r/2, r/2],
                  [r/l, -r/l]])
    
    return np.dot(H,T)


def speed_limit(motion):
    """ Clip wheel velocities """
    right_w = motion[0,0]
    left_w = motion[1,0]
    MOTION = 0.3
    
    max_abs = max(abs(right_w), abs(left_w))
    min_abs = min(abs(right_w), abs(left_w))
    
    if max_abs > 1.0:
        # Limit the peak forward velocity to 1.0
        w_r = right_w/max_abs
        w_l = left_w/max_abs
    elif min_abs < 0.3:
        # Scale velocity above 0.3
        w_r = right_w*0.3/max_abs
        w_l = left_w*0.3/max_abs
    else:
        w_r = right_w
        w_l = left_w
    
    
        
    """
    if right_w > 0:
        w_r = np.clip(right_w, MOTION, 1.0)
    else:
        w_r = np.clip(right_w, -1.0, -MOTION)
        
    if left_w > 0:
        w_l = np.clip(left_w, MOTION, 1.0)
    else:
        w_l = np.clip(left_w, -1.0, -MOTION)
    """
        
    return w_l, w_r

### Open Image Widget

In [10]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)

display(widgets.HBox([image_widget]))

width = int(image_widget.width)
height = int(image_widget.height)

In [256]:
beg_bbox_TV = [(72, 92), (136, 137)]
mid_bbox_TV = [(54, 70), (134, 121)]
mid_bbox_stool = [(175, 125), (216, 183)]
end_bbox_stool = [(199, 88), (270, 187)]

### Perform Object Detection

In [428]:
undistort_image = undistort(camera.value, mtx, dist)
detections = model(undistort_image)

"""
# Use stool as reference object
target = 62
end_target_bbox_pixel = end_bbox_stool
"""

# Use TV as reference object
target = 72
end_target_bbox_pixel = mid_bbox_TV

# draw all detections on image
for det in detections[0]:
    label = COCO_labels[det['label']]
    bbox = det['bbox']
    bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                   (int(width * bbox[2]), int(height * bbox[3]))]
    cv2.rectangle(undistort_image, bbox_pixel[0], bbox_pixel[1], (255, 0, 0), 1)
    
    print(label,det['label'], bbox_pixel)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == target]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(undistort_image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 2)
        cv2.rectangle(undistort_image, end_target_bbox_pixel[0], end_target_bbox_pixel[1], (0, 0, 255), 1)
        target_bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                   (int(width * bbox[2]), int(height * bbox[3]))]

print ("target", target_bbox_pixel)
image_widget.value = bgr8_to_jpeg(undistort_image)

laptop 72 [(75, 88), (140, 133)]
couch 62 [(170, 136), (195, 178)]
target [(75, 88), (140, 133)]


### Feature Points for Objects

From (0,0,0) to (1,0,0): 

* the TV will act as feature point from Start (0,0,0) to Midpoint (0.5,0,0).
* the Stool will act as feature point from Start (0.5,0,0) to Midpoint (1,0,0).

In [429]:
beg_feature_pts = create_feature_pts(target_bbox_pixel)
print(beg_feature_pts)
end_feature_pts = create_feature_pts(end_target_bbox_pixel)
print(end_feature_pts)

depth = 2.0
robot_theta = 0
wheel_radius = 0.0325
axle_length = 0.12
lamba = 0.05
Rtime = 0.1

error = features_errors(beg_feature_pts,end_feature_pts)
print (error.T)

UL_fp, UR_fp, LR_fp, LL_fp = beg_feature_pts
print(UL_fp)

# Generate image jacobian L
L = np.vstack((image_Jacobian(UL_fp, mtx, depth), image_Jacobian(UR_fp, mtx, depth), \
              image_Jacobian(LR_fp, mtx, depth), image_Jacobian(LL_fp, mtx, depth)))

print(image_Jacobian(UL_fp, mtx, depth))

print(L.shape)
# print(L)

J = robot_jacobian(robot_theta, wheel_radius, axle_length)
print (J)


motion = lamba*np.dot(np.linalg.pinv(np.dot(L,J)), error.T)
print(motion)
w_l, w_r = speed_limit(motion)
print(w_l, w_r)

[(75, 88), (140, 88), (140, 133), (75, 133)]
[(54, 70), (134, 70), (134, 121), (54, 121)]
[[-21]
 [-18]
 [ -6]
 [-18]
 [ -6]
 [-12]
 [-21]
 [-12]]
(75, 88)
[[ -70.97982553    0.          -36.74414537   40.99519163 -180.0023543
   -79.19170579]
 [   0.          -91.72654911  -39.5958529   217.63800066  -31.72289351
    73.48829074]]
(8, 6)
[[ 0.01625     0.01625   ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.27083333 -0.27083333]]
[[0.50809471]
 [0.5431    ]]
0.5430999952394018 0.5080947102860038


In [427]:
Rtime = 0.2

robot.set_motors(w_l, w_r)
time.sleep(Rtime)
robot.stop()